In [1]:
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load trained LSTM model
model = load_model('/content/drive/MyDrive/Machine/lstm_failure_model.h5')

# Load the MinMaxScaler used during training
scaler = joblib.load('/content/drive/MyDrive/Machine/scaler.pkl')

# Define encoding for 'Type' (machine type)
type_mapping = {'H': 0, 'L': 1, 'M': 2}
5

# {'HDF':(0), 'OSF':(1), 'PWF': (2), 'RNF':(3), 'TWF':(4), 'normal': (5)}
# Define decoding for 'Reason' (failure reasons)
reason_mapping = {
    0: 'HDF',  # Tool Wear Failure
    1: 'OSF',  # Heat Dissipation Failure
    2: 'PWF',  # Power Failure
    3: 'RNF',  # Overstrain Failure
    4: 'TWF',  # Random Failure
    5: 'Normal'  # No Failure
}

print("\n🔹 Enter machine parameters for prediction:")

# Get and encode 'Type'
while True:
    machine_type = input("🔹 Machine Type (L/H/M): ").strip().upper()
    if machine_type in type_mapping:
        machine_type_encoded = type_mapping[machine_type]
        break
    print("❌ Invalid Machine Type! Choose from: L, H, M")

# Get numerical inputs
try:
    air_temp = float(input("🌡 Air Temperature [K]: "))
    process_temp = float(input("🔥 Process Temperature [K]: "))
    rot_speed = int(input("🔄 Rotational Speed [rpm]: "))
    torque = float(input("⚙ Torque [Nm]: "))
    tool_wear = int(input("⏳ Tool Wear [min]: "))
except ValueError:
    print("❌ Invalid input! Please enter numerical values.")
    exit()

# Prepare user input (including 'Type') for scaling
user_features = np.array([[machine_type_encoded, air_temp, process_temp, rot_speed, torque, tool_wear]])

# Apply MinMaxScaler transformation (now including Type column)
user_features_scaled = scaler.transform(user_features)

# Reshape for LSTM (samples, time steps, features)
user_input_reshaped = user_features_scaled.reshape((1, 1, user_features_scaled.shape[1]))

# Model prediction
prediction = model.predict(user_input_reshaped)
predicted_reason_index = np.argmax(prediction)  # Get highest probability index

# Decode reason from mapping
predicted_reason_label = reason_mapping.get(predicted_reason_index, "Unknown")

print(f"\n✅ **Predicted Machine Wear-out Reason:** {predicted_reason_label}")
